# Análise comparativa de modelos

In [1]:
import joblib
import numpy as np
import pandas as pd
from IPython.display import display, Markdown

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

## 1. Obtenção de dados

Nessa etapa obtemos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pré-processamento.

In [2]:
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,mpg,"Eficiência do combustível do carro, medida em ...",quantitativa,contínua
1,cylinders,Número de cilindros do motor do veículo,quantitativa,discreta
2,displacement,Volume total de ar e combustível que os cilind...,quantitativa,contínua
3,horsepower,"Potência do motor, medido em cavalos de potência",quantitativa,contínua
4,weight,"Peso do veículo, medido em libras",quantitativa,contínua
5,acceleration,Tempo necessário para o veículo acelerar de 0 ...,quantitativa,contínua
6,model_year,Ano de fabricação do modelo do veículo,quantitativa,discreta
7,origin,Origem do veículo,qualitativa,nominal
8,name,Nome ou modelo do veículo,qualitativa,nominal


## 2. Preparação de dados
Aqui realizamos a normalização, codificação e o tratamento de dados discrepantes e/ou faltantes dentro do conjunto de dados.

In [3]:
X = df.drop(columns=['name'], axis=1)

target_column = 'mpg'

nominal_columns = (
    df_dict
    .query("subtipo == 'nominal'and variavel != 'name'")
    .variavel
    .to_list()
)

discrete_columns = (
    df_dict
    .query("subtipo == 'discreta'")
    .variavel
    .to_list()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'contínua' and variavel != @target_column")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

display(Markdown(
    f"- **Variável alvo:** {target_column} \n\n"
    f"- **Variáveis qualitativas nominais:** {nominal_columns} \n"
    f"- **Variáveis quantitativas discretas:** {discrete_columns} \n"
    f"- **Variáveis quantitativas contínuas:** {continuous_columns} \n"
))

- **Variável alvo:** mpg 

- **Variáveis qualitativas nominais:** ['origin'] 
- **Variáveis quantitativas discretas:** ['cylinders', 'model_year'] 
- **Variáveis quantitativas contínuas:** ['displacement', 'horsepower', 'weight', 'acceleration'] 


#### Tratamento de dados discrepantes

In [4]:
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), 
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), 
    ('normalization', StandardScaler())  
])

discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), 
    ('normalization', StandardScaler()) 
])

continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  
    ('normalization', StandardScaler()) 
])

preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('discrete', discrete_preprocessor, discrete_columns),
    ('continuous', continuous_preprocessor, continuous_columns)

])

X_preprocessed = continuous_preprocessor.fit_transform(df[continuous_columns])
model = LinearRegression()

final_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)
])

In [5]:
X_transformed = preprocessor.fit_transform(X)
X_transformed.shape

(398, 8)

In [6]:
display(X_transformed)

array([[-0.49764335,  0.77355903,  1.49819126, ...,  0.66919608,
         0.63086987, -1.29549834],
       [-0.49764335,  0.77355903,  1.49819126, ...,  1.58659918,
         0.85433297, -1.47703779],
       [-0.49764335,  0.77355903,  1.49819126, ...,  1.19342642,
         0.55047045, -1.65857724],
       ...,
       [-0.49764335,  0.77355903, -0.85632057, ..., -0.53653371,
        -0.79858454, -1.4407299 ],
       [-0.49764335,  0.77355903, -0.85632057, ..., -0.66759129,
        -0.40841088,  1.10082237],
       [-0.49764335,  0.77355903, -0.85632057, ..., -0.58895674,
        -0.29608816,  1.39128549]])

## 3. Seleção de modelos
Iremos análisar quatro modelos, que serão testados utilizando um método de validação:

* K-Nearest Neighbors
* Gradient Boosting
* Decision Tree
* Random Forest

Além disso, cada um desses algoritmos será testado com diferentes hiper-parametros, para que possamos encontrar o melhor modelo e a melhor configuração possível para esse modelo.

Utilizaremos as seguintes métricas para análise:

* **Neg Mean Squared Error (`'neg_mean_squared_error'`)**: O erro quadrático médio (MSE) é a média dos quadrados das diferenças entre os valores previstos e os valores reais. É uma medida que penaliza erros maiores de forma mais severa, já que os erros são elevados ao quadrado.
* **Neg Mean Absolute Error (`'neg_mean_absolute_error'`)**: O erro absoluto médio (MAE) é a média das diferenças absolutas entre os valores previstos e os valores reais. Ao contrário do MSE, o MAE não penaliza erros maiores mais severamente.
* **R² (`'r2'`)**: O R², ou coeficiente de determinação, mede a proporção da variabilidade total dos dados que é explicada pelo modelo. Em outras palavras, indica o quão bem os valores previstos se ajustam aos valores reais.

In [7]:
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = .2
random_state = 42
scoring = 'neg_mean_squared_error'
metrics = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']

max_iter = 1000 
models = [
    ('K-Nearest Neighbors', KNeighborsRegressor(), {
        "n_neighbors": range(3, 20, 2), 
        'weights': ['uniform', 'distance']
    }),
    ('Gradient Boosting', GradientBoostingRegressor(random_state=random_state), {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }),
     ('Decision Tree',  DecisionTreeRegressor(random_state=random_state), {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'], 
        'max_depth': [3, 6, 8]
    }),
    ('Random Forest',  RandomForestRegressor(random_state=random_state), {
        'criterion': ['squared_error', 'absolute_error'],
        'max_depth': [3, 6, 8], 
        'n_estimators': [10, 30]
    }),
]

In [8]:
results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)

for model_name, model_object, model_parameters in models:
    print(f"running {model_name}...")
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        n_jobs=-1,
        scoring=metrics
    )
    scores_df = pd.DataFrame(scores)
    scores_df['model_name'] = model_name
    results = pd.concat([results, scores_df], ignore_index=True)
    numeric_scores_df = scores_df.select_dtypes(include=['float64', 'int64'])
    scores_aggregated = numeric_scores_df.agg(['mean', 'std'])
    display(scores_aggregated)

running K-Nearest Neighbors...


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2
mean,0.545742,0.019157,-7.712959,-2.070447,0.868645
std,0.097766,0.006709,1.255142,0.169334,0.018283


running Gradient Boosting...


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2
mean,19.143982,0.012636,-7.605241,-1.994502,0.869455
std,1.695012,0.001702,0.655646,0.107781,0.018083


running Decision Tree...


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2
mean,0.431298,0.016266,-13.458341,-2.514564,0.766413
std,0.066027,0.006693,5.302292,0.336019,0.110461


running Random Forest...


,fit_time,score_time,test_neg_mean_squared_error,test_neg_mean_absolute_error,test_r2
mean,3.856865,0.014177,-7.341840,-1.954761,0.874353
std,0.544583,0.003171,1.198164,0.136663,0.022689


In [9]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(Markdown("## 3.1 Resultados gerais"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

## 3.1 Resultados gerais

O Random Forest apresentou os melhores resultados na maioria das métricas de desempenho (`test_neg_mean_squared_error`, `test_neg_mean_absolute_error` e `test_r2`), o que o torna o modelo com o melhor desempenho geral, apesar de ter um tempo de treinamento (`fit_time`) e previsão (`score_time`) um pouco mais elevados em comparação com o Decision Tree e o Gradient Boosting.

## 3.2 Persistência do modelo

In [10]:
model_name, model_object, model_parameters  = [foo for foo in models if foo[0] == "Random Forest"][0] 

model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )

approach = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model_grid_search)
])

approach.fit(X, y)

print(f"Hiper parâmetros do modelo: {approach.steps[1][1].best_params_}")

Hiper parâmetros do modelo: {'criterion': 'squared_error', 'max_depth': 8, 'n_estimators': 30}


/home/carla/avanti-bootcamp-cdd/.venv/lib/python3.10/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [11]:
joblib.dump(approach, '../models/model.joblib')

['../models/model.joblib']